In [324]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import date, timedelta, timezone, datetime
from sklearn import preprocessing

In [552]:
url = 'https://statsapi.web.nhl.com/api/v1/'
schedule = 'schedule'
date_query = 'date='
linescore_query = 'expand=schedule.linescore'
today_date = str(date.today() - timedelta(days=0,hours=5))
print(today_date)

2022-04-02


In [541]:
#get json response as text
#compose url for yesterday's schedule with linescore
url_linescore = url+schedule+'?'+linescore_query+'&'+date_query+today_date
print(url_linescore)
response = requests.get(url_linescore).text

https://statsapi.web.nhl.com/api/v1/schedule?expand=schedule.linescore&date=2022-04-02


In [542]:
dict_response = json.loads(response)

In [575]:
total_games = dict_response['dates'][0]['totalGames']

In [576]:
all_games = dict_response['dates'][0]['games']

In [577]:
home_team_scores = []
away_team_scores = []
home_teams = []
away_teams = []
goals_total = []
shots_home = []
shots_away = []
shots_total = []
game_status = []
gameId = []

for i in range(0,total_games):
    home_team_scores.append((all_games[i]['teams']['home']['score']))
    home_teams.append((all_games[i]['teams']['home']['team']['name']))
    away_team_scores.append((all_games[i]['teams']['away']['score']))
    away_teams.append((all_games[i]['teams']['away']['team']['name']))
    goals_total.append((all_games[i]['teams']['home']['score'])+(all_games[i]['teams']['away']['score']))
    shots_home.append(all_games[i]['linescore']['teams']['home']['shotsOnGoal'])
    shots_away.append(all_games[i]['linescore']['teams']['away']['shotsOnGoal'])
    shots_total.append(all_games[i]['linescore']['teams']['home']['shotsOnGoal']+all_games[i]['linescore']['teams']['away']['shotsOnGoal'])
    game_status.append(all_games[i]['linescore']['currentPeriodOrdinal'])
    gameId.append(str(all_games[i]['gamePk']))


In [578]:
#normalize goals
goals_total_np = np.array(goals_total).reshape(-1,1)

In [579]:
scaler = preprocessing.MinMaxScaler()
goals_normalized = scaler.fit_transform(goals_total_np)
goals_normalized = np.round(goals_normalized,4)

In [580]:
#prepare a list of lists for dataframe conversion
data = []
data.append(home_teams)
data.append(away_teams)
data.append(goals_total)
data.append(goals_normalized)
data.append(shots_total)
data.append(game_status)
data.append(gameId)

In [581]:
team_scores_df = pd.DataFrame(data).transpose()

In [582]:
team_scores_df.columns=['home team','away team', 'goals','normalized goals','shots total', 'game status','gameId']

In [583]:
team_scores_df.set_index('gameId')

,home team,away team,goals,normalized goals,shots total,game status
gameId,,,,,,
2021021096,Winnipeg Jets,Los Angeles Kings,0,[0.0],0,1st
2021021090,Boston Bruins,Columbus Blue Jackets,0,[0.0],0,1st
2021021091,Tampa Bay Lightning,Montréal Canadiens,0,[0.0],0,1st
2021021093,Philadelphia Flyers,Toronto Maple Leafs,0,[0.0],0,1st
2021021094,Carolina Hurricanes,Minnesota Wild,0,[0.0],0,1st
2021021098,Calgary Flames,St. Louis Blues,0,[0.0],0,1st
2021021099,San Jose Sharks,Dallas Stars,0,[0.0],0,1st
2021021092,New Jersey Devils,Florida Panthers,13,[1.0],72,OT
2021021097,Colorado Avalanche,Pittsburgh Penguins,5,[0.3846],78,3rd
